In [1]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import torch
import sys
import kaolin as kal
import matplotlib.pyplot as plt
import igl
import potpourri3d as pp3d
import trimesh
from collections import defaultdict
from copy import deepcopy

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'bookshelf'
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


In [2]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices = mesh.vertices.cpu().numpy()
faces = mesh.faces.cpu().numpy()
colors = mesh.vertex_normals.cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  5786
Number of faces:  8624


In [3]:
# Visualize mesh
# mp.plot(vertices, faces, colors)
# plt.show()

In [4]:
import json
colors_dict = {
    0: [1, 0, 0],   # Red
    1: [0, 1, 0],   # Green
    2: [0, 0, 1],   # Blue
    3: [1, 1, 0],   # Yellow
    4: [1, 0, 1],   # Magenta
    5: [0, 1, 1],   # Cyan
    6: [0.5, 0, 0], # Dark Red
    7: [0, 0.5, 0], # Dark Green
    8: [0, 0, 0.5], # Dark Blue
    9: [0.5, 0.5, 0.5] # Gray
}
colors_lst = ['Red', 'Green', 'Blue', 'Yellow', 'Magenta', 'Cyan', 'Dark Red', 'Dark Green', 'Dark Blue', 'Grey']
output_file = open('./outputs/demo/ABO/' + object + '/face_preds.json')
output = np.array(json.load(output_file))
segments = np.unique(output)
segments_idx = np.unique(output, return_inverse=True)[1]
prompts = segments.tolist()
if ('unknown' in prompts):
    prompts.remove('unknown')
# print('Prompts: ', prompts)
# colors = np.array([colors_dict[segment_id] for segment_id in segments_idx])
# for i in range(len(segments)):
#     print(colors_lst[i],':\t',segments[i])
# mp.plot(vertices, faces, colors, return_plot=True)
# plt.show()

In [5]:
n = 20
m = trimesh.Trimesh( vertices=vertices, faces=faces)
faces_adjacency = defaultdict(set)

for el in m.face_adjacency:
    u, v = el
    faces_adjacency[u].add(v)
    faces_adjacency[v].add(u)

c = []
faces_adjacency_res = deepcopy(faces_adjacency)

for k in faces_adjacency:
    for i in range(n - 1):
        start = deepcopy(faces_adjacency_res[k])
        end = set(deepcopy(faces_adjacency_res[k]))
        for f in start:
            end.update(faces_adjacency[f])
        faces_adjacency_res[k] = end
    c.append(len(faces_adjacency_res[k]))
c = np.array(c)

In [6]:
print(faces_adjacency_res[0])

{0, 1, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 1228, 1229, 1230, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 1106, 1107, 86, 1108, 1109, 84, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 1130, 1131, 1132, 109, 1133, 1134, 1135, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 1153, 1154, 1155, 1156, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 1182, 1183, 1231, 165, 166, 167, 168, 169, 170, 171, 172, 173, 1190, 1191, 1194, 1195, 1196, 1197, 1198, 181, 182, 183, 184, 185, 186, 187, 188, 189, 1205, 1214, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 1

In [7]:
# colors = mesh.vertex_normals.cpu().numpy()
i = 0
colors = np.clip(np.random.rand(faces.shape[0], 3), 0, 0.2)
# colors = np.clip(np.random.rand(faces.shape[0], 3), 0, 1)
colors[i] = np.array([255,0,0])
for f in faces_adjacency_res[i]:
    colors[f] = np.array([255,0,0])
p = mp.plot(vertices, faces, colors, return_plot=True)
# p.add_points(vertices[faces[i, 0]].reshape(1,-1), 
#             shading={'point_color':'red', 'point_size':0.3})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.4272670…

In [8]:
adj = igl.adjacency_matrix(faces)
cc = igl.connected_components(adj)
print('Number of connected components:', cc[0])
mp.plot(vertices, faces, cc[1])
plt.show()

Number of connected components: 158


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.4272670…

In [9]:
mp.plot(vertices, faces, np.random.rand(faces.shape[0]))
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.4272670…

In [10]:
# Gaussian Curvature
k = igl.gaussian_curvature(vertices, faces)
# print('Gaussian Curvature')
# mp.plot(vertices, faces, k)
# plt.show()

In [11]:
# query_pt = np.array([0.03,0.03,1.25])
query_pt = np.array([0.4,0.47,0.25])
# i = np.argmin(np.sum((vertices-query_pt)**2, axis = 1))
i = np.random.randint(vertices.shape[0])
i = 7
solver = pp3d.MeshHeatMethodDistanceSolver(
    vertices,
    faces
)
distances = solver.compute_distance(i)
p = mp.plot(vertices, faces, c = np.cos(100*distances), return_plot=True)
# p = mp.plot(vertices, faces, c = distances, return_plot=True)
p.add_points(vertices[i].reshape(1, -1), shading = {'point_color':'red', 'point_size':0.2})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.4272670…

In [17]:
n_samples = int(vertices.shape[0] * 2)
trimeshMesh = trimesh.Trimesh(vertices, faces)
pts = trimesh.sample.sample_surface_even(trimeshMesh, n_samples)[0]
mp.plot(pts, shading={'point_size':0.04})

c:\ProgramData\miniforge3\envs\meshseg\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.809986…

In [18]:
np.random.seed(420)
n_samples = int(vertices.shape[0] * 2)
trimeshMesh = trimesh.Trimesh(vertices, faces)
pts = trimesh.sample.sample_surface_even(trimeshMesh, n_samples)[0]
# rand_pt = np.random.randint(pts.shape[0])
rand_pt = 9921
pts_solver = pp3d.PointCloudHeatSolver(pts)
distances = pts_solver.compute_distance(rand_pt)
p = mp.plot(pts, c=np.cos(70*distances), shading={'point_size':0.08}, return_plot=True)
p.add_points(pts[rand_pt].reshape(1,-1), shading = {'point_color':'red', 'point_size':0.35})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.954155…

IndexError: index 9921 is out of bounds for axis 0 with size 1546

In [ ]:
print(pts[rand_pt])
query_pt = np.array([-0.279844, 0.4,-0.13167364])
print(np.argmin(np.sum((query_pt - pts)**2, axis=1)))

[-0.26501321  0.5296835   0.02254017]
10962


In [ ]:
pts_solver = pp3d.PointCloudHeatSolver(vertices)
distances = pts_solver.compute_distance(i)
p = mp.plot(vertices, faces, c = distances, return_plot=True, shading = {'point_size':0.05})
p.add_points(vertices[i].reshape(1, -1), shading = {'point_color':'red', 'point_size':0.2})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.36…

In [ ]:
pts_solver = pp3d.PointCloudHeatSolver(vertices)
distances = pts_solver.compute_distance(i)
p = mp.plot(vertices, return_plot=True, shading = {'point_size':0.05})
# p.add_points(vertices[i].reshape(1, -1), shading = {'point_color':'red', 'point_size':0.2})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.36…